In [ ]:
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import numpy as np
import joblib
import traceback


In [ ]:
# 모델 및 스케일러 로드
model = joblib.load('./model/one_class_svm_model.joblib')
scaler = joblib.load('./model/scaler.joblib')

In [ ]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 1

voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

# 센서 보정 값
ZMPT_offset = 2.554
ZMPT_scale = 1000.0

ACS712_offset = 2.508
ACS712_sensitivity = 0.1


In [ ]:
# 피처 생성 함수
def extract_features(voltage_seq, current_seq):
    voltage = np.mean(voltage_seq)
    current = np.mean(current_seq)
    voltage_diff = abs(voltage_seq[-1] - voltage_seq[-2])
    current_diff = abs(current_seq[-1] - current_seq[-2])
    voltage_ma = np.mean(voltage_seq[-5:])
    current_ma = np.mean(current_seq[-5:])
    power = voltage * current
    power_diff = abs(power - (voltage_seq[-2] * current_seq[-2]))
    return [voltage, current, voltage_diff, current_diff,
            voltage_ma, current_ma, power, power_diff]

In [ ]:
# 부하 안정화 여부 체크 변수
load_stable = False
stable_counter = 0
STABLE_THRESHOLD = 100  # 정상 상태로 간주할 연속 횟수

In [12]:
# 실시간 측정
voltage_buffer = []
current_buffer = []

while True:
    try:
        # 센서 원시값 측정
        v_raw = voltage_ch.voltage
        c_raw = current_ch.voltage

        # 보정값 계산
        voltage = (v_raw - ZMPT_offset) * ZMPT_scale
        current = (c_raw - ACS712_offset) / ACS712_sensitivity

        voltage_buffer.append(voltage)
        current_buffer.append(current)

        if len(voltage_buffer) > 5:
            voltage_buffer.pop(0)
            current_buffer.pop(0)

            features = extract_features(voltage_buffer, current_buffer)
            features_scaled = scaler.transform([features])
            prediction = model.predict(features_scaled)[0]  # 1이면 이상

            if prediction == -1:
                stable_counter = 0  # 스파이크 발생 시 초기화
                if load_stable:
                    print("[🔴] 아크 발생 (모델 이상 탐지)")
            else:
                stable_counter += 1
                if stable_counter >= STABLE_THRESHOLD:
                    load_stable = True
                print("[🟢] 정상")

    except Exception as e:
        if load_stable:
            print("[⚠️] I2C 오류 감지됨: 아크 가능성 높음")
        else:
            print("[ℹ️] 초기 부하 중 I2C 오류 무시")
        traceback.print_exc()
        continue

    time.sleep(1 / 860.0)  # 860Hz 주기

[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[ℹ️] 초기 부하 중 I2C 오류 무시
[ℹ️] 초기 부하 중 I2C 오류 무시
[ℹ️] 초기 부하 중 I2C 오류 무시


Traceback (most recent call last):
  File "/tmp/ipykernel_3358/1769241783.py", line 8, in <module>
    v_raw = voltage_ch.voltage
            ^^^^^^^^^^^^^^^^^^
  File "/home/raspberry/powerenv/lib/python3.11/site-packages/adafruit_ads1x15/analog_in.py", line 64, in voltage
    volts = self.convert_to_voltage(self.value)
                                    ^^^^^^^^^^
  File "/home/raspberry/powerenv/lib/python3.11/site-packages/adafruit_ads1x15/analog_in.py", line 59, in value
    return self._ads.read(pin)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/raspberry/powerenv/lib/python3.11/site-packages/adafruit_ads1x15/ads1x15.py", line 337, in read
    return self._read(pin)
           ^^^^^^^^^^^^^^^
  File "/home/raspberry/powerenv/lib/python3.11/site-packages/adafruit_ads1x15/ads1x15.py", line 369, in _read
    while not self._conversion_complete():
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/raspberry/powerenv/lib/python3.11/site-packages/adafruit_ads1x15/ads1x15.py", line

[ℹ️] 초기 부하 중 I2C 오류 무시


Traceback (most recent call last):
  File "/tmp/ipykernel_3358/1769241783.py", line 9, in <module>
    c_raw = current_ch.voltage
            ^^^^^^^^^^^^^^^^^^
  File "/home/raspberry/powerenv/lib/python3.11/site-packages/adafruit_ads1x15/analog_in.py", line 64, in voltage
    volts = self.convert_to_voltage(self.value)
                                    ^^^^^^^^^^
  File "/home/raspberry/powerenv/lib/python3.11/site-packages/adafruit_ads1x15/analog_in.py", line 59, in value
    return self._ads.read(pin)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/raspberry/powerenv/lib/python3.11/site-packages/adafruit_ads1x15/ads1x15.py", line 337, in read
    return self._read(pin)
           ^^^^^^^^^^^^^^^
  File "/home/raspberry/powerenv/lib/python3.11/site-packages/adafruit_ads1x15/ads1x15.py", line 369, in _read
    while not self._conversion_complete():
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/raspberry/powerenv/lib/python3.11/site-packages/adafruit_ads1x15/ads1x15.py", line

[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상
[🟢] 정상


KeyboardInterrupt: 